In [24]:
from CwnGraph import CwnBase, CwnSense

In [3]:
import igraph as ig

In [35]:
from tqdm.autonotebook import tqdm

/Users/seantyh/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [6]:
cwn = CwnBase()

In [8]:
all_chars = cwn.find_lemma("^.$")

In [13]:
char_nodes = set(x.lemma for x in all_chars)

In [19]:
from itertools import chain
sense_nodes = list(set(chain.from_iterable(x.senses for x in all_chars)))

In [55]:
sense_nodes[0].relations[0]

('is_synset', <CwnSynset[syn_014828]: 治理杭州的地方政府。>, 'forward')

In [56]:
sense_relations = [(x.id, r[1].id, r[0]) for x in sense_nodes for r in x.relations if type(r[1]) is CwnSense and r[2]=="forward"]

In [57]:
len(sense_relations)

30664

In [101]:
G = ig.Graph(directed=False)

for s in tqdm(sense_nodes):
    G.add_vertex(s.id)
vs = set(x["name"] for x in G.vs)
for r in tqdm(sense_relations):
    src_id, tgt_id, rel_type = r
    if tgt_id in vs:
        G.add_edge(src_id, tgt_id, relation=rel_type)

In [62]:
G.summary()

'IGRAPH UN-- 21713 34331 -- \n+ attr: name (v), relation (e)'

In [102]:
def find_sense_paths(c1, c2):
    c1_senses = [x.id for x in cwn.find_lemma(f"^{c1}$")]
    c2_senses = [x.id for x in c2.senses]
    senses_path = [G.get_shortest_paths(c1_x, c2_senses) for c1_x in c1_senses]
    return senses_path

In [69]:
from mesh_import import mesh

In [91]:
import pandas as pd
from opencc import OpenCC
import numpy as np
cc = OpenCC('s2t')

In [74]:
m2 = pd.read_csv(mesh.get_data_dir() / "cns/m2_cld.csv", index_col=0)

In [79]:
tw_words = [cc.convert(x) for x in m2["word"]]

In [96]:
dists = []
for x in tqdm(tw_words):
    try:
        dists.append(len(G.get_shortest_paths(x[0], x[1])[0]))
    except:
        dists.append(np.nan)

/Users/seantyh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Couldn't reach some vertices at /Users/travis/build/igraph/python-igraph/vendor/source/igraph/src/structural_properties.c:768
  after removing the cwd from sys.path.


In [89]:
G.get_shortest_paths(tw_words[0][0], tw_words[0][1])

[[1894,
  7512,
  14730,
  8953,
  8955,
  13728,
  10897,
  10893,
  1232,
  16615,
  16619,
  3861,
  6697,
  21140,
  445]]